In [2]:
import pandas as pd
import polars as pl
import os

In [3]:
machine_MILL_path = r"Machine_#2_MILL\Machine_#2_MILL"
machine_MILL_days = os.listdir(machine_MILL_path)

# Charge les data de MILL de chaque jour dans un dict
data_MILL = {}
for file in machine_MILL_days :
    file_path = os.path.join(machine_MILL_path, file)
    data_MILL[file] = pl.read_parquet(file_path)
    data_MILL[file] = data_MILL[file].drop("time")


In [4]:
print(data_MILL)

{'13-03-2025_10h35.parquet': shape: (3_177_584, 85)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ stSigAxCu ┆ stSigAxCu ┆ stSigAxCu ┆ stSigAxCu ┆ … ┆ stSigAxPo ┆ stSigAxPo ┆ stSigPowe ┆ __index_ │
│ rrentB    ┆ rrentC    ┆ rrentX    ┆ rrentY    ┆   ┆ werY      ┆ werZ      ┆ rMotS     ┆ level_0_ │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ _        │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ i64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ null      ┆ null      ┆ null      ┆ 0.623217  ┆ … ┆ null      ┆ null      ┆ null      ┆ 0        │
│ 0.305296  ┆ 0.322347  ┆ 0.180628  ┆ null      ┆ … ┆ 0.989401  ┆ 6.314389  ┆ null      ┆ 1        │
│ null      ┆ null      ┆ null      ┆ 0

In [5]:
def combine_first_columns_from_csv(directory):
    """
    Lit tous les fichiers CSV dans un dossier, extrait la première colonne de chacun,
    et les combine en un seul DataFrame avec le nom des fichiers comme colonnes.
    """
    combined_df = pd.DataFrame()  # DataFrame vide pour stocker les données

    for file in os.listdir(directory):
        if file.endswith(".csv"):  # Vérifier si c'est un fichier CSV
            file_path = os.path.join(directory, file)
            
            # Lire seulement la première colonne du fichier
            try :
                df = pd.read_csv(file_path, usecols=[1],sep=";") 
            except :
                print(file_path)
                break

            # Renommer la colonne avec le nom du fichier sans extension
            column_name = os.path.splitext(file)[0]
            df.columns = [column_name]

            # Concaténer avec le DataFrame principal (axis=1 pour fusionner en colonnes)
            combined_df = pd.concat([combined_df, df], axis=1)
        
    print("folder done")

    return combined_df

In [6]:
machine_MECATIS_path = r"Machine_#1_Mecatis\Machine_#1_Mecatis"
machine_MECATIS_days = os.listdir(machine_MECATIS_path)

data_MECATIS = {}
for index, folder in enumerate(machine_MECATIS_days) :
    folder_path = os.path.join(machine_MECATIS_path, folder)
    data_MECATIS[folder] = combine_first_columns_from_csv(folder_path)


folder done
folder done
folder done
folder done
folder done
folder done


In [7]:
print(data_MECATIS)

{'Warmup_Mecatis_03_02_25':          diCodeMesure  lrSigAxeBTemp  lrSigAxeCTemp  lrSigAxeXTemp  \
0                 0.0           10.0           10.0            0.0   
1                 0.0           10.0           10.0            0.0   
2                 0.0           10.0           10.0            0.0   
3                 0.0           10.0           10.0            0.0   
4                 0.0           10.0           10.0            0.0   
...               ...            ...            ...            ...   
2065384           NaN            NaN            NaN            NaN   
2065385           NaN            NaN            NaN            NaN   
2065386           NaN            NaN            NaN            NaN   
2065387           NaN            NaN            NaN            NaN   
2065388           NaN            NaN            NaN            NaN   

         lrSigAxeYTemp  lrSigAxeZTemp  lrSigCryoLubState  \
0                  0.0            0.0                0.0   
1          

In [8]:
def check_df(df):
    # Afficher les premières lignes
    print("HEAD")
    print(df.head(), "\n")

    # Voir les infos du DataFrame
    print("Info")
    print(df.info(), "\n")

In [9]:

def get_machine_means_dict(machine_dict : dict):
    means_dict = {}
    for key, df in machine_dict.items(): 
        means_dict[key]=df.mean()
        means_dict[key].name = key
        print(f"means {key} done")
    print("DONE")
    return means_dict

def get_machine_vars_dict(machine_dict : dict):
    vars_dict = {}
    for key, df in machine_dict.items(): 
        vars_dict[key]=df.var()
        vars_dict[key].name = key
        print(f"vars {key} done")
    print("DONE")
    return vars_dict

In [10]:
MECATIS_means=get_machine_means_dict(data_MECATIS)
MECATIS_vars=get_machine_vars_dict(data_MECATIS)


means Warmup_Mecatis_03_02_25 done
means Warmup_Mecatis_05_02_25 done
means Warmup_Mecatis_10_03_25 done
means Warmup_Mecatis_10_03_25_3 done
means Warmup_Mecatis_10_03_25_4 done
means Warmup_Mecatis_10_03_25_5 done
DONE
vars Warmup_Mecatis_03_02_25 done
vars Warmup_Mecatis_05_02_25 done
vars Warmup_Mecatis_10_03_25 done
vars Warmup_Mecatis_10_03_25_3 done
vars Warmup_Mecatis_10_03_25_4 done
vars Warmup_Mecatis_10_03_25_5 done
DONE


In [11]:
MILL_means=get_machine_means_dict(data_MILL)
MILL_vars=get_machine_vars_dict(data_MILL)

means 13-03-2025_10h35.parquet done
means 13-03-2025_7h35.parquet done
means 19-02-2025_9h20.parquet done
DONE
vars 13-03-2025_10h35.parquet done
vars 13-03-2025_7h35.parquet done
vars 19-02-2025_9h20.parquet done
DONE


In [12]:
#debug

print(type(MILL_means['13-03-2025_10h35.parquet']))
for i in MECATIS_means.values():
    print(i.shape)
    print(type(i))
    print(i.name)

print("\n")
for key, df in MILL_means.items():
    df = df.to_pandas().transpose().squeeze()
    df.name = key
    print(df.shape)
    print(type(df))
    print(df.name)

<class 'polars.dataframe.frame.DataFrame'>
(74,)
<class 'pandas.core.series.Series'>
Warmup_Mecatis_03_02_25
(74,)
<class 'pandas.core.series.Series'>
Warmup_Mecatis_05_02_25
(74,)
<class 'pandas.core.series.Series'>
Warmup_Mecatis_10_03_25
(74,)
<class 'pandas.core.series.Series'>
Warmup_Mecatis_10_03_25_3
(74,)
<class 'pandas.core.series.Series'>
Warmup_Mecatis_10_03_25_4
(71,)
<class 'pandas.core.series.Series'>
Warmup_Mecatis_10_03_25_5


(85,)
<class 'pandas.core.series.Series'>
13-03-2025_10h35.parquet
(85,)
<class 'pandas.core.series.Series'>
13-03-2025_7h35.parquet
(85,)
<class 'pandas.core.series.Series'>
19-02-2025_9h20.parquet


In [13]:
MECATIS_mean_list = [i for i in MECATIS_means.values()]
MILL_mean_list = []
for key, df in MILL_means.items():
    df = df.to_pandas().transpose().squeeze()
    df.name = key
    MILL_mean_list.append(df)


df_means = pd.concat(MECATIS_mean_list + MILL_mean_list, axis=1)

MECATIS_var_list = [i for i in MECATIS_vars.values()]
MILL_var_list = []
for key, df in MILL_vars.items():
    df = df.to_pandas().transpose().squeeze()
    df.name = key
    MILL_var_list.append(df)

df_vars = pd.concat(MECATIS_var_list + MILL_var_list, axis=1)


In [15]:
df_means.to_excel("means.xlsx")
df_vars.to_excel("variances.xlsx")